In [23]:
import pandas as pd
import os 

input_data_dir = os.getcwd().replace('/notebooks', '') + '/data/external'
output_data_dir = os.getcwd().replace('/notebooks', '') + '/data/interim'

team_filename = 'game_summary_by_team.csv'
indiv_player_game_stats_filename = 'game_summary_by_player.csv'
indiv_player_metrics_filename = 'player_metadata.csv'

team_df = pd.read_csv(os.path.join(input_data_dir, team_filename))
indiv_player_game_df = pd.read_csv(os.path.join(input_data_dir, indiv_player_game_stats_filename))
team_df.tail()

games = team_df["url"].unique()

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
def get_prevwk_team_df(team_df, favorite, underdog, season, week, prev_wks):
    
    game_teams_df = pd.DataFrame()
    
    fave_games_df = get_indiv_team_prevwk_df(team_df, favorite, season, week, prev_wks)
    dog_games_df = get_indiv_team_prevwk_df(team_df, underdog, season, week, prev_wks)
    
    game_teams_df = game_teams_df.append(fave_games_df, ignore_index=True)
    game_teams_df = game_teams_df.append(dog_games_df, ignore_index=True)
        
    return game_teams_df

def get_indiv_team_prevwk_df(team_df, team, season, week, num_wks_to_find):
    week = int(week)
    indiv_team_prevwk_df = pd.DataFrame()
    num_wks_found = 0
    
    print("{} / {} / {}".format(team, season, week))
    for prev_wk in range(week - 1, 0, -1):
        
        #valid_row = get_valid_row(team_df_subset, prev_wk, season)
        team_game_df, num_wks_found = append_team_prevwk_game(team_df, team, season, week, prev_wk, num_wks_found)
        if team_game_df is not None:
            indiv_team_prevwk_df = indiv_team_prevwk_df.append(team_game_df, ignore_index=True)

        if num_wks_found == num_wks_to_find:
            break
    
    if num_wks_found < prev_wks:
        for prev_wk in range(17, 0, -1):
            team_game_df, num_wks_found = append_team_prevwk_game(team_df, team, season - 1, week, prev_wk, num_wks_found)
            
            if team_game_df is not None:
                indiv_team_prevwk_df = indiv_team_prevwk_df.append(team_game_df, ignore_index=True)

            if num_wks_found == num_wks_to_find:
                break
    
    return indiv_team_prevwk_df

def append_team_prevwk_game(team_df, team, season, week, prev_wk, num_wks_found):
    print("  {} / {} / {}".format(team, season, prev_wk))
    team_df_subset = team_df[team_df["week"] == str(prev_wk)]
    team_df_subset = team_df_subset[team_df_subset["season"] == season]
    #print(team_df_subset["team"].unique())
    valid_row_flg = True if team in team_df_subset["team"].unique() else False
    
    #print(valid_row_flg)
    if valid_row_flg == True:
        returned_df = team_df_subset[team_df_subset["team"] == team]
        num_wks_found += 1
    else:
        returned_df = None
        
    return returned_df, num_wks_found

In [25]:
prev_wks = 3
output_df = pd.DataFrame()
for game_link in games[-100:-90]:

    game_df = team_df[team_df["url"] == game_link]
    favorite = game_df[game_df["favorite_flg"] == True]["team"].values[0]
    underdog = game_df[game_df["favorite_flg"] == False]["team"].values[0]
    season = game_df[game_df["favorite_flg"] == True]["season"].values[0]
    week = game_df[game_df["favorite_flg"] == True]["week"].values[0]
    line = abs(game_df[game_df["favorite_flg"] == True]["line"].values[0])
    over_under = abs(game_df[game_df["favorite_flg"] == True]["over_under"].values[0])
    favorite_pts = game_df[game_df["favorite_flg"] == True]["off_pts"].values[0]
    underdog_pts = game_df[game_df["favorite_flg"] == False]["off_pts"].values[0]
    home_favorite = game_df[game_df["favorite_flg"] == True]["home_flg"].values[0]
    
    spread_diff = favorite_pts - line - underdog_pts 
    favorite_cover_flg = True if spread_diff > 0 else False
    push_flg = True if spread_diff == 0 else False
    underdog_cover_flg = True if spread_diff < 0 else False
    
    game_dict = {
        "game_link" : game_link,
        "season" : season,
        "week" : week,
        "favorite" : favorite,
        "favorite_pts" : favorite_pts,
        "underdog" : underdog,
        "underdog_pts" : underdog_pts,
        "line" : line,
        "over_under" : over_under,
        "home_favorite": home_favorite,
        "spread_diff" : spread_diff,
        "favorite_cover_flg" : favorite_cover_flg,
        "push_flg" : push_flg,
        "underdog_cover_flg" : underdog_cover_flg
    }
    
    teams_df_subset = get_prevwk_team_df(team_df, favorite, underdog, season, week, prev_wks)
    
    for key, val in game_dict.items():
        game_dict[key] = [val]
    df_to_append = pd.DataFrame(data=game_dict)
    output_df = output_df.append(df_to_append, ignore_index=True)
#utput_df

Denver Broncos / 2017 / 13
  Denver Broncos / 2017 / 12
  Denver Broncos / 2017 / 11
  Denver Broncos / 2017 / 10
Miami Dolphins / 2017 / 13
  Miami Dolphins / 2017 / 12
  Miami Dolphins / 2017 / 11
  Miami Dolphins / 2017 / 10
Atlanta Falcons / 2017 / 13
  Atlanta Falcons / 2017 / 12
  Atlanta Falcons / 2017 / 11
  Atlanta Falcons / 2017 / 10
Minnesota Vikings / 2017 / 13
  Minnesota Vikings / 2017 / 12
  Minnesota Vikings / 2017 / 11
  Minnesota Vikings / 2017 / 10
Green Bay Packers / 2017 / 13
  Green Bay Packers / 2017 / 12
  Green Bay Packers / 2017 / 11
  Green Bay Packers / 2017 / 10
Tampa Bay Buccaneers / 2017 / 13
  Tampa Bay Buccaneers / 2017 / 12
  Tampa Bay Buccaneers / 2017 / 11
  Tampa Bay Buccaneers / 2017 / 10
Los Angeles Chargers / 2017 / 13
  Los Angeles Chargers / 2017 / 12
  Los Angeles Chargers / 2017 / 11
  Los Angeles Chargers / 2017 / 10
Cleveland Browns / 2017 / 13
  Cleveland Browns / 2017 / 12
  Cleveland Browns / 2017 / 11
  Cleveland Browns / 2017 / 10
Los 

In [26]:
teams_df_subset

,url,team,season,week,opponent,line,over_under,off_pts,def_pts_allowed,home_flg,...,opp_dl_tackles_solo_sum,opp_dl_snaps_sum,opp_dl_tackles_per_snap,opp_k_fga_sum,opp_k_fgm_sum,opp_k_xpa_sum,opp_k_xpm_sum,opp_k_fg_pct,opp_p_punt_sum,opp_p_punt_yds_per_punt_sum
0,/boxscores/201712030atl.htm,Atlanta Falcons,2017,13,Minnesota Vikings,-2.0,48.5,9,14,True,...,6.0,211.0,0.145455,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/boxscores/201711260atl.htm,Atlanta Falcons,2017,12,Tampa Bay Buccaneers,-10.5,47.0,34,20,True,...,10.0,227.0,0.166667,NaN,NaN,NaN,NaN,1.00,NaN,NaN
2,/boxscores/201711200sea.htm,Atlanta Falcons,2017,11,Seattle Seahawks,-1.0,46.0,34,31,False,...,10.0,249.0,0.258065,NaN,NaN,NaN,NaN,0.75,NaN,NaN
3,/boxscores/201712030nor.htm,New Orleans Saints,2017,13,Carolina Panthers,-5.5,48.0,31,21,True,...,8.0,262.0,0.208955,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201711260ram.htm,New Orleans Saints,2017,12,Los Angeles Rams,2.5,54.0,20,26,False,...,7.0,135.0,0.218182,NaN,NaN,NaN,NaN,0.80,NaN,NaN
5,/boxscores/201711190nor.htm,New Orleans Saints,2017,11,Washington Redskins,-9.5,52.5,34,31,True,...,3.0,165.0,0.057971,NaN,NaN,NaN,NaN,1.00,NaN,NaN


In [30]:
cols = ["url", "team", "season", "opponent", "team_sacks_allowed", "opp_sacks_allowed"]
team_df[(team_df["team"] == "Minnesota Vikings") & \
        (team_df["season"] == 2017) & \
        (team_df["week"].isin(['12','11','10']))
       ][cols]

,url,team,season,opponent,team_sacks_allowed,opp_sacks_allowed
2944,/boxscores/201711120was.htm,Minnesota Vikings,2017,Washington Redskins,0,1
2968,/boxscores/201711190min.htm,Minnesota Vikings,2017,Los Angeles Rams,0,2
2992,/boxscores/201711230det.htm,Minnesota Vikings,2017,Detroit Lions,2,3


In [29]:
favorite_prev_wk_df = teams_df_subset[teams_df_subset["team"] == favorite]
row_key = "row_key"
df_col_key = "df_col_key"
idx_col_key = "idx_col_name"
indices_dict = [
    {row_key: "off_pts", df_col_key: "def_pts_allowed", idx_col_key: "off_pts_idx"},
    {row_key: "def_pts_allowed", df_col_key: "off_pts", idx_col_key: "off_pts_allowed_idx"},
    {row_key: "team_total_yds", df_col_key: "opp_total_yds", idx_col_key: "off_total_yds_idx"},
    {row_key: "opp_total_yds", df_col_key: "team_total_yds", idx_col_key: "total_yds_allowed_idx"},
    {row_key: "team_first_downs", df_col_key: "opp_first_downs", idx_col_key: "off_first_downs_idx"},
    {row_key: "opp_first_downs", df_col_key: "team_first_downs", idx_col_key: "first_downs_allowed_idx"},
    {row_key: "team_rush_yds", df_col_key: "opp_rush_yds", idx_col_key: "off_rush_yds_idx"},
    {row_key: "opp_rush_yds", df_col_key: "team_rush_yds", idx_col_key: "rush_yds_allowed_idx"},
    {row_key: "team_pass_yds", df_col_key: "opp_pass_yds", idx_col_key: "off_pass_yds_idx"},
    {row_key: "opp_pass_yds", df_col_key: "team_pass_yds", idx_col_key: "pass_yds_allowed_idx"},
    {row_key: "opp_sacks_allowed", df_col_key: "team_sacks_allowed", idx_col_key: "sacks_idx"},
    {row_key: "team_sacks_allowed", df_col_key: "opp_sacks_allowed", idx_col_key: "sacks_allowed_idx"},
    {row_key: "team_penalty_yds", df_col_key: "opp_penalty_yds", idx_col_key: "penalty_yds_idx"},
    {row_key: "opp_penalty_yds", df_col_key: "team_penalty_yds", idx_col_key: "opp_penalty_yds_idx"},
    {row_key: "team_turnovers", df_col_key: "opp_turnovers", idx_col_key: "turnovers_idx"},
    {row_key: "opp_turnovers", df_col_key: "team_turnovers", idx_col_key: "turnovers_forced_idx"}
]
for idx, row in favorite_prev_wk_df[0:1].iterrows():
    fave_pts = float(row["off_pts"])
    opp_df = get_indiv_team_prevwk_df(team_df, row["opponent"], row["season"], row["week"], prev_wks)
    avg_opp_pts_allowed = float(opp_df["def_pts_allowed"].mean())
    off_pts_idx = fave_pts / avg_opp_pts_allowed

teams_df_subset[]

Minnesota Vikings / 2017 / 13
  Minnesota Vikings / 2017 / 12
  Minnesota Vikings / 2017 / 11
  Minnesota Vikings / 2017 / 10


,url,team,season,opponent,team_sacks_allowed,opp_sacks_allowed
0,/boxscores/201712030atl.htm,Atlanta Falcons,2017,Minnesota Vikings,0,2
1,/boxscores/201711260atl.htm,Atlanta Falcons,2017,Tampa Bay Buccaneers,0,1
2,/boxscores/201711200sea.htm,Atlanta Falcons,2017,Seattle Seahawks,1,3
3,/boxscores/201712030nor.htm,New Orleans Saints,2017,Carolina Panthers,2,2
4,/boxscores/201711260ram.htm,New Orleans Saints,2017,Los Angeles Rams,3,4
5,/boxscores/201711190nor.htm,New Orleans Saints,2017,Washington Redskins,2,2


In [16]:
opp_df

,url,team,season,week,opponent,line,over_under,off_pts,def_pts_allowed,home_flg,...,opp_dl_tackles_solo_sum,opp_dl_snaps_sum,opp_dl_tackles_per_snap,opp_k_fga_sum,opp_k_fgm_sum,opp_k_xpa_sum,opp_k_xpm_sum,opp_k_fg_pct,opp_p_punt_sum,opp_p_punt_yds_per_punt_sum
0,/boxscores/201711230det.htm,Minnesota Vikings,2017,12,Detroit Lions,-2.5,46.0,30,23,False,...,9.0,275.0,0.189189,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,/boxscores/201711190min.htm,Minnesota Vikings,2017,11,Los Angeles Rams,-1.0,46.0,24,7,True,...,5.0,212.0,0.118421,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201711120was.htm,Minnesota Vikings,2017,10,Washington Redskins,-1.0,43.0,38,30,False,...,6.0,151.0,0.177419,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [19]:
off_pts_idx

0.45

In [20]:
for col in opp_df.columns:
    print(col)

url
team
season
week
opponent
line
over_under
off_pts
def_pts_allowed
home_flg
winner_flg
favorite_flg
team_first_downs
opp_first_downs
team_rush_att
opp_rush_att
team_rush_yds
opp_rush_yds
team_rush_tds
opp_rush_tds
team_pass_comp
opp_pass_comp
team_pass_att
opp_pass_att
team_pass_yds
opp_pass_yds
team_pass_tds
opp_pass_tds
team_pass_int
opp_pass_int
team_sacks
opp_sacks
team_penalty_yds
opp_penalty_yds
team_net_pass_yds
opp_net_pass_yds
team_total_yds
opp_total_yds
team_fumbles
opp_fumbles
team_fumbles_lost
opp_fumbles_lost
team_turnovers
opp_turnovers
team_penalties
opp_penalties
team_third_down_conv
opp_third_down_conv
team_fourth_down_conv
opp_fourth_down_conv
team_time_of_poss
opp_time_of_poss
team_rush_le
opp_rush_le
team_rush_yds_le
opp_rush_yds_le
team_rush_td_le
opp_rush_td_le
team_rush_lt
opp_rush_lt
team_rush_yds_lt
opp_rush_yds_lt
team_rush_td_lt
opp_rush_td_lt
team_rush_lg
opp_rush_lg
team_rush_yds_lg
opp_rush_yds_lg
team_rush_td_lg
opp_rush_td_lg
team_rush_md
opp_rush_md